In [1]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
import pandas as pd

file_path = '/content/drive/My Drive/modelpred.csv'
df = pd.read_csv(file_path)

df.head()

<ipython-input-2-a1f2f7fe5d56>:4: DtypeWarning: Columns (51,52,53) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


,p_match,inns,bat,p_bat,team_bat,bowl,p_bowl,team_bowl,ball,ball_id,...,bat_out,wagonX,wagonY,wagonZone,line,length,shot,control,predscore,wprob
0,1001349,1,Aaron Finch,5334,Australia,Lasith Malinga,49758,Sri Lanka,1,0.01,...,True,208,178,3,ON_THE_STUMPS,SHORT_OF_A_GOOD_LENGTH,DEFENDED,0.0,-1,-1.0
1,1001349,1,Aaron Finch,5334,Australia,Lasith Malinga,49758,Sri Lanka,2,0.02,...,True,243,175,3,ON_THE_STUMPS,GOOD_LENGTH,DEFENDED,1.0,-1,-1.0
2,1001349,1,Aaron Finch,5334,Australia,Lasith Malinga,49758,Sri Lanka,3,0.03,...,True,189,242,4,ON_THE_STUMPS,SHORT_OF_A_GOOD_LENGTH,DEFENDED,0.0,-1,-1.0
3,1001351,1,Aaron Finch,5334,Australia,Lasith Malinga,49758,Sri Lanka,3,0.03,...,True,166,194,5,OUTSIDE_OFFSTUMP,SHORT_OF_A_GOOD_LENGTH,DEFENDED,1.0,-1,-1.0
4,1001351,1,Aaron Finch,5334,Australia,Lasith Malinga,49758,Sri Lanka,4,0.04,...,True,15,222,6,OUTSIDE_OFFSTUMP,GOOD_LENGTH,CUT_SHOT,1.0,-1,-1.0


In [3]:

df = pd.read_csv(file_path)
df['date'] = pd.to_datetime(df['date'])
#to check till what date we have ball by ball data of
last_date = df['date'].max()

print(f"The last date in the 'date' column is: {last_date}")

<ipython-input-3-15dbc3a031c2>:1: DtypeWarning: Columns (51,52,53) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


The last date in the 'date' column is: 2024-08-21 00:00:00


In [4]:
#using three variables we will predict the expected runs, we will group them in a way that for each unique combination of these three variables we will take the mean of batruns scored, so that if any time the same unique combination of these contextual variables occur we can predtict the expected runs -  Ensure necessary columns are in the correct datatype
df['inns_rr'] = pd.to_numeric(df['inns_rr'], errors='coerce')
df['inns_balls_rem'] = pd.to_numeric(df['inns_balls_rem'], errors='coerce')
df['inns_wkts'] = pd.to_numeric(df['inns_wkts'], errors='coerce')
df['batruns'] = pd.to_numeric(df['batruns'], errors='coerce')

# Group by the unique combinations of 'inns_rr', 'inns_balls_rem', and 'inns_wkts'
grouped = df.groupby(['inns_rr', 'inns_balls_rem', 'inns_wkts'])['batruns'].mean().reset_index()

# Rename the 'batruns' column to 'XRuns' in the grouped DataFrame
grouped.rename(columns={'batruns': 'XRuns'}, inplace=True)

# Merge the grouped DataFrame back into the original DataFrame
df = df.merge(grouped, on=['inns_rr', 'inns_balls_rem', 'inns_wkts'], how='left')

# Display the first few rows of the updated DataFrame
print(df.head())

# Save the updated DataFrame back to CSV
new_file_path = '/content/drive/My Drive/t20_bbb_with_XRuns.csv'
df.to_csv(new_file_path, index=False)

print("The DataFrame with the 'XRuns' column has been successfully saved to:", new_file_path)

   p_match  inns          bat  p_bat   team_bat            bowl  p_bowl  \
0  1001349     1  Aaron Finch   5334  Australia  Lasith Malinga   49758   
1  1001349     1  Aaron Finch   5334  Australia  Lasith Malinga   49758   
2  1001349     1  Aaron Finch   5334  Australia  Lasith Malinga   49758   
3  1001351     1  Aaron Finch   5334  Australia  Lasith Malinga   49758   
4  1001351     1  Aaron Finch   5334  Australia  Lasith Malinga   49758   

   team_bowl  ball  ball_id  ... wagonX  wagonY  wagonZone              line  \
0  Sri Lanka     1     0.01  ...    208     178          3     ON_THE_STUMPS   
1  Sri Lanka     2     0.02  ...    243     175          3     ON_THE_STUMPS   
2  Sri Lanka     3     0.03  ...    189     242          4     ON_THE_STUMPS   
3  Sri Lanka     3     0.03  ...    166     194          5  OUTSIDE_OFFSTUMP   
4  Sri Lanka     4     0.04  ...     15     222          6  OUTSIDE_OFFSTUMP   

                   length      shot  control  predscore  wprob     X

In [5]:
# File path
file_path = '/content/drive/My Drive/t20_bbb_with_XRuns.csv'

# Load the CSV file
df = pd.read_csv(file_path)

# Ensure the 'date' column is in datetime format
df['date'] = pd.to_datetime(df['date'])

# Filter for Suryakumar Yadav
suryakumar_df = df[df['bat'] == 'Suryakumar Yadav']

# Filter for the years 2023 and 2024
suryakumar_recent = suryakumar_df[(suryakumar_df['date'].dt.year == 2023) | (suryakumar_df['date'].dt.year == 2024)]

# Calculate the total sum of XRuns
total_XRuns = suryakumar_recent['XRuns'].sum()

print(f"Total XRuns for Suryakumar Yadav in 2023 and 2024: {total_XRuns}")

<ipython-input-5-ef6016e4d5e6>:5: DtypeWarning: Columns (51,52,53) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


Total XRuns for Suryakumar Yadav in 2023 and 2024: 1725.041645109846


In [6]:
# File path
file_path = '/content/drive/My Drive/t20_bbb_with_XRuns.csv'

# Load the CSV file
df = pd.read_csv(file_path)

# Ensure necessary columns are in the correct datatype
df['inns_rr'] = pd.to_numeric(df['inns_rr'], errors='coerce')
df['inns_balls_rem'] = pd.to_numeric(df['inns_balls_rem'], errors='coerce')
df['inns_wkts'] = pd.to_numeric(df['inns_wkts'], errors='coerce')
df['bowlruns'] = pd.to_numeric(df['bowlruns'], errors='coerce')

# Group by the unique combinations of 'inns_rr', 'inns_balls_rem', and 'inns_wkts'
grouped = df.groupby(['inns_rr', 'inns_balls_rem', 'inns_wkts'])['bowlruns'].mean().reset_index()

# Rename the 'batruns' column to 'XRunsb' in the grouped DataFrame
grouped.rename(columns={'bowlruns': 'XRunsb'}, inplace=True)

# Merge the grouped DataFrame back into the original DataFrame
df = df.merge(grouped, on=['inns_rr', 'inns_balls_rem', 'inns_wkts'], how='left')

# Display the first few rows of the updated DataFrame
print(df.head())

# Save the updated DataFrame back to CSV
new_file_path = '/content/drive/My Drive/t20_bbb_with_XRunsb.csv'
df.to_csv(new_file_path, index=False)

print("The DataFrame with the 'XRunsb' column has been successfully saved to:", new_file_path)

<ipython-input-6-ab3386d0254e>:5: DtypeWarning: Columns (51,52,53) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


   p_match  inns          bat  p_bat   team_bat            bowl  p_bowl  \
0  1001349     1  Aaron Finch   5334  Australia  Lasith Malinga   49758   
1  1001349     1  Aaron Finch   5334  Australia  Lasith Malinga   49758   
2  1001349     1  Aaron Finch   5334  Australia  Lasith Malinga   49758   
3  1001351     1  Aaron Finch   5334  Australia  Lasith Malinga   49758   
4  1001351     1  Aaron Finch   5334  Australia  Lasith Malinga   49758   

   team_bowl  ball  ball_id  ... wagonY  wagonZone              line  \
0  Sri Lanka     1     0.01  ...    178          3     ON_THE_STUMPS   
1  Sri Lanka     2     0.02  ...    175          3     ON_THE_STUMPS   
2  Sri Lanka     3     0.03  ...    242          4     ON_THE_STUMPS   
3  Sri Lanka     3     0.03  ...    194          5  OUTSIDE_OFFSTUMP   
4  Sri Lanka     4     0.04  ...    222          6  OUTSIDE_OFFSTUMP   

                   length      shot  control  predscore  wprob     XRuns  \
0  SHORT_OF_A_GOOD_LENGTH  DEFENDED     

In [7]:
# Calculate the RAAR (batruns - XRuns)
df['RAAR'] = df['batruns'] - df['XRuns']

# Assuming you have an 'XRunsb' column for expected bowling runs
# Calculate the RBAR (bowlruns - XRunsb)
df['RBAR'] = df['bowlruns'] - df['XRuns']  # Using 'XRuns' since 'XRunsb' isn't explicitly created

# Display the first few rows of the updated DataFrame
print(df.head())

# Save the updated DataFrame back to CSV
new_file_path = '/content/drive/My Drive/t20_bbb_with_XRuns_RAAR_RBAR.csv'
df.to_csv(new_file_path, index=False)

print("The DataFrame with 'RAAR' and 'RBAR' columns has been successfully saved to:", new_file_path)

   p_match  inns          bat  p_bat   team_bat            bowl  p_bowl  \
0  1001349     1  Aaron Finch   5334  Australia  Lasith Malinga   49758   
1  1001349     1  Aaron Finch   5334  Australia  Lasith Malinga   49758   
2  1001349     1  Aaron Finch   5334  Australia  Lasith Malinga   49758   
3  1001351     1  Aaron Finch   5334  Australia  Lasith Malinga   49758   
4  1001351     1  Aaron Finch   5334  Australia  Lasith Malinga   49758   

   team_bowl  ball  ball_id  ...              line                  length  \
0  Sri Lanka     1     0.01  ...     ON_THE_STUMPS  SHORT_OF_A_GOOD_LENGTH   
1  Sri Lanka     2     0.02  ...     ON_THE_STUMPS             GOOD_LENGTH   
2  Sri Lanka     3     0.03  ...     ON_THE_STUMPS  SHORT_OF_A_GOOD_LENGTH   
3  Sri Lanka     3     0.03  ...  OUTSIDE_OFFSTUMP  SHORT_OF_A_GOOD_LENGTH   
4  Sri Lanka     4     0.04  ...  OUTSIDE_OFFSTUMP             GOOD_LENGTH   

       shot control  predscore  wprob     XRuns    XRunsb      RAAR      RBAR  


#Calculating Top Prolific Battes Who Have Defied The Average Replacement Of Batters


In [8]:

df['batruns'] = pd.to_numeric(df['batruns'], errors='coerce')
df['XRuns'] = pd.to_numeric(df['XRuns'], errors='coerce')
df['RAAR'] = pd.to_numeric(df['RAAR'], errors='coerce')


grouped_bat = df.groupby('bat').agg(
    total_batruns=('batruns', 'sum'),
    total_XRuns=('XRuns', 'sum'),
    total_RAAR=('RAAR', 'sum'),
    first_date=('date', 'min'),
    last_date=('date', 'max'),
    team_bat_entries=('team_bat', lambda x: ', '.join(x.unique()))
).reset_index()

# Filter players with at least 5000 total batruns
filtered_bat = grouped_bat[grouped_bat['total_batruns'] >= 5000]

# Display the filtered table
print(filtered_bat)


                   bat  total_batruns   total_XRuns  total_RAAR  first_date  \
1       AB de Villiers           5829   4960.329933  868.670067  2015-04-11   
25         Aaron Finch           7677   7577.181597   99.818403  2015-01-03   
334         Alex Hales           9488   9150.369075  337.630925  2015-01-02   
490     Andre Fletcher           5605   5755.933100 -150.933100  2015-01-14   
495      Andre Russell           6951   5994.360757  956.639243  2015-01-03   
862         Babar Azam          10099  10207.567462 -108.567462  2015-09-08   
1280       Chris Gayle           7721   6988.996953  732.003047  2015-01-09   
1287        Chris Lynn           7087   6576.331208  510.668792  2015-01-02   
1343      Colin Ingram           6713   6164.412508  548.587492  2015-05-15   
1344       Colin Munro           9010   8703.071218  306.928782  2015-05-29   
1485      David Miller           7116   6883.877259  232.122741  2015-01-09   
1493      David Warner           7133   6905.082799 

In [10]:
import plotly.express as px

# Ensure 'control' and 'wide' columns are in numeric format
df['control'] = pd.to_numeric(df['control'], errors='coerce')
df['wide'] = pd.to_numeric(df['wide'], errors='coerce')

# Filter rows where 'wide' column is 0
df_filtered = df[df['wide'] == 0]

# Group by 'bat' and calculate total sum of 'control' where control is 0, and count rows
grouped_control = df_filtered.groupby('bat').agg(
    total_batruns=('batruns', 'sum'),
    total_XRuns=('XRuns', 'sum'),
    total_RAAR=('RAAR', 'sum'),
    total_control_0=('control', lambda x: (x == 0).sum()),  # Sum of control entries that are 0
    total_rows=('bat', 'count'),  # Count of total rows for each 'bat'
    first_date=('date', 'min'),
    last_date=('date', 'max'),
    team_bat_entries=('team_bat', lambda x: ', '.join(x.unique()))
).reset_index()

# Filter players with at least 5000 total batruns
filtered_control = grouped_control[grouped_control['total_batruns'] >= 5000]

# Display the filtered table
print(filtered_control)

# Save the filtered table to a new CSV
output_file_path_control = '/content/drive/My Drive/batsmen_with_5000_plus_control_and_rows.csv'
filtered_control.to_csv(output_file_path_control, index=False)

print("The table for batsmen with 5000+ runs and control calculations has been saved to:", output_file_path_control)

# Visualization using Plotly (Horizontal Bar Chart to show prolific batters based on RAAR)
fig = px.bar(filtered_control,
             x='total_RAAR',
             y='bat',
             text='total_RAAR',
             title="Top Batters by RAAR with Control Calculations",
             labels={"total_RAAR": "Total RAAR", "bat": "Batter"},
             orientation='h')

# Show plot
fig.show()


                   bat  total_batruns  total_XRuns   total_RAAR  \
1       AB de Villiers           5829  4827.312214  1001.687786   
25         Aaron Finch           7677  7320.564725   356.435275   
333         Alex Hales           9488  8935.580584   552.419416   
489     Andre Fletcher           5605  5572.339333    32.660667   
494      Andre Russell           6951  5709.134576  1241.865424   
861         Babar Azam          10099  9944.854280   154.145720   
1279       Chris Gayle           7721  6651.717857  1069.282143   
1286        Chris Lynn           7087  6401.209654   685.790346   
1342      Colin Ingram           6713  5985.660213   727.339787   
1343       Colin Munro           9010  8452.788029   557.211971   
1484      David Miller           7116  6678.199693   437.800307   
1492      David Warner           7133  6683.418733   449.581267   
1498       Dawid Malan           7614  7470.369813   143.630187   
1555      Devon Conway           5335  5386.516369   -51.51636

In [11]:
import plotly.graph_objects as go

# Create an interactive table using Plotly
fig_table = go.Figure(data=[go.Table(
    header=dict(values=list(filtered_control.columns),
                fill_color='paleturquoise',
                align='left'),
    cells=dict(values=[filtered_control[col] for col in filtered_control.columns],
               fill_color='lavender',
               align='left'))
])

# Update layout for better readability
fig_table.update_layout(
    title="Interactive Table for Batsmen with 5000+ Runs",
    height=600,  # You can adjust the height as needed
    width=1000,  # You can adjust the width as needed
)

# Show the table
fig_table.show()


In [13]:
# Visualization using Plotly (Vertical Bar Chart with bat on x-axis and RAAR on y-axis)
fig = px.bar(filtered_control,
             x='bat',  # Set bat column on the x-axis
             y='total_RAAR',  # Set RAAR on the y-axis
             text='total_RAAR',
             title="Top Batters by RAAR with Control Calculations",
             labels={"total_RAAR": "Total RAAR", "bat": "Batter"})

# Update layout to rotate x-axis tick labels for vertical display
fig.update_layout(
    xaxis_title="Batter",
    yaxis_title="Total RAAR",
    title_font_size=20,
    height=600,  # Adjust height as necessary
    xaxis_tickangle=-90  # Rotate x-axis labels to vertical
)

# Show the plot
fig.show()


In [14]:
import plotly.express as px


df['control'] = pd.to_numeric(df['control'], errors='coerce')
df['wide'] = pd.to_numeric(df['wide'], errors='coerce')


df_filtered = df[df['wide'] == 0]


grouped_false_shot = df_filtered.groupby('bat').agg(
    total_control_0=('control', lambda x: (x == 0).sum()),  # Count control = 0
    total_occurrences=('bat', 'count'),
    total_RAAR=('RAAR', 'sum')
).reset_index()

# Calculate False Shot Percentage
grouped_false_shot['false_shot_percentage'] = (grouped_false_shot['total_control_0'] / grouped_false_shot['total_occurrences']) * 100

# Scatter plot between False Shot Percentage and RAAR
fig = px.scatter(grouped_false_shot,
                 x='false_shot_percentage',
                 y='total_RAAR',
                 text='bat',
                 title="Scatter Plot: False Shot Percentage vs RAAR",
                 labels={"false_shot_percentage": "False Shot Percentage (%)", "total_RAAR": "Total RAAR"})

# Update layout for better readability
fig.update_layout(
    title_font_size=20,
    xaxis_title="False Shot Percentage (%)",
    yaxis_title="Total RAAR",
    height=600,  # Adjust as necessary
)

# Show plot
fig.show()


In [15]:
import plotly.express as px

# Ensure 'control' and 'wide' columns are in numeric format
df['control'] = pd.to_numeric(df['control'], errors='coerce')
df['wide'] = pd.to_numeric(df['wide'], errors='coerce')
df['batruns'] = pd.to_numeric(df['batruns'], errors='coerce')

# Filter rows where 'wide' column is 0
df_filtered = df[df['wide'] == 0]


grouped_bat = df_filtered.groupby('bat').agg(
    total_batruns=('batruns', 'sum'),
    total_control_0=('control', lambda x: (x == 0).sum()),  # Count control = 0
    total_occurrences=('bat', 'count'),  # Count total rows
    total_RAAR=('RAAR', 'sum')  # Sum of RAAR
).reset_index()


grouped_bat_filtered = grouped_bat[grouped_bat['total_batruns'] >= 5000]


grouped_bat_filtered['false_shot_percentage'] = (grouped_bat_filtered['total_control_0'] / grouped_bat_filtered['total_occurrences']) * 100

fig = px.scatter(grouped_bat_filtered,
                 x='false_shot_percentage',
                 y='total_RAAR',
                 text='bat',
                 title="Scatter Plot: False Shot Percentage vs RAAR for Batters with 5000+ Runs",
                 labels={"false_shot_percentage": "False Shot Percentage (%)", "total_RAAR": "Total RAAR"})


fig.update_layout(
    title_font_size=20,
    xaxis_title="False Shot Percentage (%)",
    yaxis_title="Total RAAR",
    height=600,
)

# Show plot
fig.show()


<ipython-input-15-41eae610a816>:23: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

